## Using ANN on CarPricesData

In [1]:
# Reading the cleaned numeric car prices data
import pandas as pd
import numpy as np

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

CarPricesDataNumeric=pd.read_pickle('CarPricesDataNumeric.pkl')
CarPricesDataNumeric.head()

,Price,Age,KM,HP,MetColor,Automatic,CC,Doors,Weight,FuelType_CNG,FuelType_Diesel,FuelType_Petrol
0,13500,23.0,46986,90,1,0,2000.0,3,1165.0,0,1,0
1,13750,23.0,72937,90,1,0,2000.0,3,1165.0,0,1,0
2,13950,24.0,41711,90,1,0,2000.0,3,1165.0,0,1,0
3,14950,26.0,48000,90,0,0,2000.0,3,1165.0,0,1,0
4,13750,30.0,38500,90,0,0,2000.0,3,1170.0,0,1,0


## Splitting the Data into Training and Testing

In [2]:
# After Standardization of data
# Separate Target Variable and Predictor Variables
TargetVariable=['Price']
Predictors=['Age', 'KM', 'HP', 'MetColor', 'Automatic', 'CC', 'Doors',
       'Weight', 'FuelType_CNG', 'FuelType_Diesel', 'FuelType_Petrol']

X=CarPricesDataNumeric[Predictors].values
y=CarPricesDataNumeric[TargetVariable].values


### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)


# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(999, 11)
(999, 1)
(429, 11)
(429, 1)


In [4]:
# Standardized predictors
X[0:3]

array([[-1.79400224, -0.58190964, -0.76628391,  0.69377792, -0.24361277,
         2.34607519, -1.08412485,  2.10364977, -0.10976426,  2.92982809,
        -2.75724675],
       [-1.79400224,  0.11238127, -0.76628391,  0.69377792, -0.24361277,
         2.34607519, -1.08412485,  2.10364977, -0.10976426,  2.92982809,
        -2.75724675],
       [-1.73978236, -0.72303656, -0.76628391,  0.69377792, -0.24361277,
         2.34607519, -1.08412485,  2.10364977, -0.10976426,  2.92982809,
        -2.75724675]])

## Regression using Deep Learning- Artificial Neural Networks(ANN)

In [6]:
from keras.models import Sequential
from keras.layers import Dense

In [7]:
# create ANN model
model = Sequential()

# Defining the first layer of the model
model.add(Dense(units=5, input_dim=11, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam' )

In [8]:
# Fitting the ANN to the Training set
#1000/20 --- forward/backward
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 1.0272
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 1.0007
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8869
Epoch 4/50
50/50 [==============================] - 0s 3ms/step - loss: 0.6806
Epoch 5/50
50/50 [==============================] - 0s 3ms/step - loss: 0.4782
Epoch 6/50
50/50 [==============================] - 0s 3ms/step - loss: 0.3121
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2071
Epoch 8/50
50/50 [==============================] - 0s 3ms/step - loss: 0.1621
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1446
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1355
Epoch 11/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1301
Epoch 12/50
50/50 [==============================] - 0s 3ms/step - loss: 0.1265
Epoch 13/50
50/50 [==============================

## Finding best set of parameters using manual grid search

In [9]:
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=10, input_dim=11, kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=5, input_dim=20, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = 10, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)

In [10]:
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 77.85569913987956
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 75.7328690477642
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 83.38735127107353
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 80.05009547949366
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 79.28604945746625
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 75.98055047525604
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 78.08903960547843
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 77.9135164886054
9 Parameters: batch_size: 15 - epochs: 5 Accuracy: 74.66979387305793
10 Parameters: batch_size: 15 - epochs: 10 Accuracy: 71.46772697207092
11 Parameters: batch_size: 15 - epochs: 50 Accuracy: 75.64297644987745
12 Parameters: batch_size: 15 - epochs: 100 Accuracy: 77.52281092929894
13 Parameters: batch_size: 20 - epochs: 5 Accuracy: 73.84871759168998
14 Parameters: batch_size: 20 - epochs: 10 Accuracy: 72.98378059654917
15 Parameters: batch_s

### Generating the model using best hyperparameters

In [11]:
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 5, epochs = 50, verbose=0)

In [12]:
MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
print('Accuracy:', 100-MAPE)

Accuracy: 84.37112508700126


### Generating the predictions on Testing Data

In [13]:
# Predictions on testing data
Predictions=model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)

# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Price']=y_test_orig
TestingData['PredictedPrice']=Predictions
TestingData.head()

,Age,KM,HP,MetColor,Automatic,CC,Doors,Weight,FuelType_CNG,FuelType_Diesel,FuelType_Petrol,Price,PredictedPrice
0,20.0,35000.0,97.0,1.0,0.0,1400.0,5.0,1110.0,0.0,0.0,1.0,16500.0,16832.839844
1,76.0,89520.0,110.0,1.0,0.0,1600.0,3.0,1050.0,0.0,0.0,1.0,8950.0,7955.057617
2,56.0,66777.0,110.0,0.0,0.0,1600.0,5.0,1090.0,0.0,0.0,1.0,11950.0,10991.720703
3,39.0,24723.0,110.0,1.0,1.0,1600.0,4.0,1060.0,0.0,0.0,1.0,12695.0,13172.583008
4,68.0,35000.0,110.0,1.0,0.0,1600.0,3.0,1050.0,0.0,0.0,1.0,8900.0,9397.404297
